In [ ]:
import os
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import umap
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import LabelBinarizer
from sklearn import metrics


In [ ]:

cpdir = 'Checkpoints/Snail_Redo_for_metrics/best-2019-08-16-10-24/'
checkpoint = 'best-checkpoint-9200'
if not cpdir.endswith('/'):
    cpdir += '/'

In [ ]:
sval = cpdir.split("/")
valdir = "/".join(sval[0:-2]) + "/"
val_images = np.load(valdir + 'validation_images.npy')
val_labels = np.load(valdir + 'validation_labels.npy')
val_nums = val_labels.argmax(axis=1)

In [ ]:
### load the Image 3C clusters into a dataframe
df = pd.read_csv('Data/Snail/ClusterIDs.csv')
#print(df.head())

# map classes to zero based ids
rmin = df.ClusterID.min()
rmax = df.ClusterID.max()
rmap = {}
for i, rc in enumerate(np.arange(rmin, rmax + 1, 1)):
    rmap[rc] = i
    
df['Image3C_Cluster'] = df.ClusterID.map(rmap)
### read in the fdl coordinates, not all of the cells have fdl coords
fdl = pd.read_csv('Data/Snail/FDL_coords.csv', delimiter=';')
fdl['Y'] = -fdl['Y']
#print(fdl.head())

### join the fdl and df for a dataframe

#res.head()


In [ ]:
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71", "orange", "yellow", "green"]
#sns.palplot(sns.color_palette(flatui))

def scatter(df, hue=None, legend_title=None):

    #hue = 'Image3C_Cluster'
    fig =plt.figure(figsize=(8,8))
    ax = fig.add_subplot(1,1,1)
    g = sns.scatterplot(x='X', y='Y', hue=hue, s=4,
                    alpha=.9, edgecolor=None,
                    palette='Set1',
                    legend='full', data=df, ax=ax)

    plt.legend(loc='upper left', prop={'size':10}, bbox_to_anchor=(1,1), frameon=False)
    #plt.legend(frameon=False)
       
    

In [ ]:
mm_images = np.memmap('Data/images.mm', dtype=np.float32, mode='r')
images = mm_images.reshape((-1, 64, 64, 5))
images = images[:,16:48,16:48,[0,2,4]]
del mm_images
images.shape

In [ ]:
mm_labels = np.memmap('Data/labels.mm', dtype=np.float32, mode='r')
labels = mm_labels.reshape((images.shape[0], -1))
del mm_labels
labels.shape

In [ ]:
imean = images.mean(axis=(1,2), keepdims=True)
istd = images.std(axis=(1,2), keepdims=True)
images =(images - imean)/istd

In [ ]:
cpdir = 'Checkpoints/Snail_Redo_for_metrics/best-2019-08-16-09-02/'
checkpoint = 'best-checkpoint-10800'

In [ ]:
sess = tf.Session()
saver = tf.train.import_meta_graph(cpdir + checkpoint + '.meta')
saver.restore(sess, cpdir + checkpoint)

softmax = sess.graph.get_tensor_by_name('Softmax:0')
batch = sess.graph.get_tensor_by_name('Placeholder:0')

In [ ]:
vsm = sess.run(softmax, feed_dict={batch:images})
vsm_nums = vsm.argmax(axis=1)

df['predicted'] = vsm_nums

In [ ]:
nums = labels.argmax(axis=1)

metrics.accuracy_score(nums, vsm_nums), metrics.accuracy_score(nums, df['Image3C_Cluster'])


In [ ]:
print(metrics.classification_report(nums, vsm_nums))

In [ ]:
max_sm = vsm.max(axis=1)
df['max_sm'] = max_sm

In [ ]:
df['match'] = df.predicted == df['Image3C_Cluster']

In [ ]:
def cbin(x):
    return(int(10*x))

df['cbin'] = df.max_sm.map(cbin)/10

In [ ]:
res = fdl.set_index('EventID').join(df.set_index('EventID'))

In [ ]:
plt.figure(figsize=(8,8))
scatter(res, hue='predicted')
plt.savefig('C9-plots/predicted.svg', bbox_inches='tight')

In [ ]:
res0 = res[res.max_sm > .8]
resx = res[res.max_sm < .5]
len(res0), len(resx)

In [ ]:
scatter(res0, hue='predicted')
plt.savefig('C9-plots/predicted_high_prob.svg', bbox_inches='tight')

In [ ]:
scatter(resx, hue='predicted')
plt.savefig('C9-plots/predicted_low_prob.svg', bbox_inches='tight')

In [ ]:
fig =plt.figure(figsize=(8,8))

sns.scatterplot(x='X', y='Y', hue='cbin',s=6, #size='max_sm',
                data=res[res.match == True], alpha=.9, legend='full',
                palette=sns.color_palette("Blues", len(res.cbin.unique())),
                edgecolor=None)

plt.legend(loc='upper left', prop={'size':10}, bbox_to_anchor=(1,1), frameon=False)

plt.savefig('C9-plots/matched_true_prob.svg', bbox_inches='tight')


In [ ]:
fig =plt.figure(figsize=(8,8))

sns.scatterplot(x='X', y='Y', hue='cbin',s=6, #size='max_sm',
                data=res[res.match == False], alpha=.9, legend='full',
                palette=sns.color_palette("Greens", len(res.cbin.unique())),
                edgecolor=None)

plt.legend(loc='upper left', prop={'size':10}, bbox_to_anchor=(1,1), frameon=False)

plt.savefig('C9-plots/matched_false_prob.svg', bbox_inches='tight')


In [ ]:
fig =plt.figure(figsize=(8,8))

sns.scatterplot(x='X', y='Y', hue='predicted',s=6, #size='max_sm',
                data=res[res.match == True], alpha=.9, legend='full',
                palette='Set1',
                edgecolor=None)

plt.legend(loc='upper left', prop={'size':10}, bbox_to_anchor=(1,1), frameon=False)

plt.savefig('C9-plots/matched_true_class.svg', bbox_inches='tight')


In [ ]:
fig =plt.figure(figsize=(8,8))

sns.scatterplot(x='X', y='Y', hue='predicted',s=6, #size='max_sm',
                data=res[res.match == False], alpha=.9, legend='full',
                palette='Set1',
                edgecolor=None)

plt.legend(loc='upper left', prop={'size':10}, bbox_to_anchor=(1,1), frameon=False)

plt.savefig('C9-plots/matched_false_class.svg', bbox_inches='tight')


In [ ]:
testlossdf = pd.read_csv('Loss_acc_plots/Snail9-2/run-test-tag-loss.csv')
trainlossdf = pd.read_csv('Loss_acc_plots/Snail9-2/run-train-tag-loss.csv')
testaccdf = pd.read_csv('Loss_acc_plots/Snail9-2/run-test-tag-accuracy_1.csv')
trainaccdf = pd.read_csv('Loss_acc_plots/Snail9-2/run-train-tag-accuracy_1.csv')

In [ ]:
plt.plot(trainlossdf['Step'], trainlossdf['Value'], label="Training", color='C0')
plt.plot(testlossdf['Step'], testlossdf['Value'], label="Test", color='C1')

plt.xlim(-200, 12000)
plt.xlabel("Iteration step")
plt.ylabel("Loss")
legend = plt.legend(loc='upper left', prop={'size':10},
                    bbox_to_anchor=(.25,1), framealpha=0.0)
sns.despine()
plt.savefig('C9-plots/training_loss.svg', bbox_inches='tight')


In [ ]:
plt.plot(testaccdf['Step'], testaccdf['Value'], label='Test')
plt.plot(trainaccdf['Step'], trainaccdf['Value'], label='Training')
plt.xlim(-200, 12000)
plt.xlabel("Iteration step")
plt.ylabel("Accuracy")
plt.legend(framealpha=0.0)
sns.despine()
plt.savefig('C9-plots/training_accuracy.svg', bbox_inches='tight')
